In [1]:
"""
author: EdgardoCS @FSU Jena
date: 16.04.2025
"""

import pandas as pd
import pingouin as pg
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.anova import anova_lm
from statsmodels.formula.api import ols, mixedlm
from statsmodels.stats.multicomp import pairwise_tukeyhsd

pd.options.mode.chained_assignment = None

In [2]:
input_data = "output/data_sorted.xlsx"
columns = ['id', 'points', 'gender', 'segment', 'location', 'type']

data = pd.read_excel(input_data, usecols=columns)

# focus on female and male for now
data = (data[data['gender'].isin(['female', 'male'])])

# Conditions:
# front + self + female
# front + self + male
# front + other + female
# front + other + male
#
# back + self + female
# back + self + male
# back + other + female
# back + other + male

In [3]:
data['segment'] = data['segment'].astype('category')
data['location'] = data['location'].astype('category')
data['type'] = data['type'].astype('category')
data['gender'] = data['gender'].astype('category')
data['id'] = data['id'].astype('category')  # Random effect

In [4]:

model = smf.mixedlm(
    "points ~ segment * location * type * gender",
    data,
    groups=data["id"]
)
result = model.fit()
print(result.summary())

                                   Mixed Linear Model Regression Results
Model:                              MixedLM                 Dependent Variable:                 points     
No. Observations:                   26915                   Method:                             REML       
No. Groups:                         2409                    Scale:                              0.9005     
Min. group size:                    1                       Log-Likelihood:                     -39005.2520
Max. group size:                    36                      Converged:                          Yes        
Mean group size:                    11.2                                                                   
-----------------------------------------------------------------------------------------------------------
                                                                Coef.  Std.Err.    z    P>|z| [0.025 0.975]
---------------------------------------------------------------

In [5]:
# Subset example: armpit + male
# subset = data[(data['segment'] == 'armpit') & (data['gender'] == 'male')]
#
# # Pairwise t-test
# posthoc = pg.pairwise_tests(
#     dv='points',
#     within='type',
#     subject='id',
#     data=subset,
#     padjust='bonferroni',
#     effsize='cohen'
# )
# print(posthoc)

In [6]:
# # 1. Is there any difference between female and male when smelling others?
# target = (data[
#               (data['type'] == 'other') &
#               (data['gender'].isin(['female', 'male']))]
#           .copy())
# three_model = ols("""points ~ C(segment) + C(gender) + C(location) +
#                C(segment):C(gender) + C(segment):C(location) + C(gender):C(location) +
#                C(segment):C(gender):C(location)""", data=target).fit()
# res1 = anova_lm(three_model, typ=2)
#
# # C(segment):C(gender) -> p= 0,35
# # C(segment):C(location) -> p= 7,38
# # C(gender):C(location) -> p= 0,22
# # C(segment):C(gender):C(location) -> p= 0,49
#
# # Answer, apparently *None* given the three model calculation

In [7]:
# # 2. Is there any difference between female and male when smelling themselves?
# target = (data[
#               (data['type'] == 'self') &
#               (data['gender'].isin(['female', 'male']))]
#           .copy())
# three_model = ols("""points ~ C(segment) + C(gender) + C(location) +
#                C(segment):C(gender) + C(segment):C(location) + C(gender):C(location) +
#                C(segment):C(gender):C(location)""", data=target).fit()
# res2 = anova_lm(three_model, typ=2)
#
# # C(gender) -> p= 0,009
# # C(segment):C(gender) -> p= 0,97
# # C(segment):C(location) -> p= 0,02
# # C(gender):C(location) -> p= 0,66
# # C(segment):C(gender):C(location) -> p= 0,81
#
# # Answer, there is a main effect of gender, and there is significant difference
# # between segment and location when participants smell themselves

In [8]:
# # Take both genders, front and back when smelling themselves
# target = data[
#     (data['location'].isin(['front', 'back'])) &
#     (data['type'] == 'self') &
#     (data['gender'].isin(['female', 'male']))
#     ].copy()
# tukey = pairwise_tukeyhsd(endog=target['points'],
#                           # groups=target['location'] + target['segment'],
#                           groups=target['gender'] + target['segment'],
#                           alpha=0.05)
#
# # group     meandiff	p-adj	lower	upper	reject
# # (front vs back)
# # armpit	-0.0481	    0.9999	-0.2054	0.1093	False
# # chest	    -0.2054	    0.7353	-0.5271	0.1163	False
# # feet	    0.0671	    0.9986	-0.1164	0.2506	False
# # hair	    -0.0627	    1.0	    -0.3866	0.2612	False
# # hand	    -0.0428	    1.0	    -0.281	0.1954	False
# # knee	    -0.0394	    1.0	    -0.597	0.5182	False
# # mouth	    -0.3381	    0.1214	-0.7076	0.0314	False
# # neck	    0.0695	    1.0	    -0.224	0.3629	False
# # pelvis	0.0039	    1.0	    -0.1752	0.1831	False
#
# # group     meandiff	p-adj	lower	upper	reject
# # (female vs male)
# # armpit	-0.084	    0.9009	-0.236	0.0679	False
# # chest	    -0.0897	    0.9999	-0.4026	0.2231	False
# # feet	    -0.075	    0.9957	-0.2619	0.1119	False
# # hair	    -0.0351	    1.0	    -0.3826	0.3125	False
# # hand	    -0.0119	    1.0	    -0.2541	0.2302	False
# # knee	    -0.0618	    1.0	    -0.6508	0.5272	False
# # mouth	    -0.0487	    1.0	    -0.3572	0.2598	False
# # neck	    -0.0892	    0.9999	-0.3832	0.2048	False
# # pelvis	-0.0146	    1.0	    -0.1944	0.1652	False


In [9]:
# # 3. is there any difference when males smell themselves vs when they smell others?
# target = (data[
#               (data['type'].isin(['self', 'other'])) &
#               (data['gender'] == 'male')]
#           .copy())
# three_model = ols("""points ~ C(segment) + C(type) + C(location) +
#                C(segment):C(type) + C(segment):C(location) + C(type):C(location) +
#                C(segment):C(type):C(location)""", data=target).fit()
# res3 = anova_lm(three_model, typ=2)
#
# # C(type) -> p= 0,031
# # C(segment):C(type) -> p= 0,53
# # C(segment):C(location) -> p= 0,001
# # C(type):C(location) -> p= 0,88
# # C(segment):C(type):C(location) -> p= 0,98
#
# # Answer, there is a main effect of type, and there is significant difference
# # between segment and location when participants smell themselves vs when they smell others

In [10]:
# # Lets check the above
# target = data[
#     (data['location'].isin(['front', 'back'])) &
#     (data['gender'] == 'male') &
#     (data['type'].isin(['self', 'other']))
#     ].copy()
# tukey = pairwise_tukeyhsd(endog=target['points'],
#                           # groups=target['location'] + target['segment'],
#                           groups=target['type'] + target['segment'],
#                           alpha=0.05)
#
# # group     meandiff	p-adj	lower	upper	reject
# # (front vs back)
# # armpit	-0.1062	    0.8087	-0.2815	0.0691	False
# # chest	    -0.3709	    0.0032	-0.6774	-0.0643	True
# # feet	    0.0838	    0.9961	-0.1266	0.2942	False
# # hair	    -0.1236	    0.999	-0.4705	0.2233	False
# # hand	    -0.0699	    1.0	    -0.3747	0.2348	False
# # knee	    -0.0048	    1.0	    -0.5552	0.5456	False
# # mouth	    -0.1502	    0.9962	-0.5281	0.2277	False
# # neck	    0.11	    0.9898	-0.1436	0.3636	False
# # pelvis	-0.0228	    1.0	    -0.2266	0.1811	False
#
# # group     meandiff	p-adj	lower	upper	reject
# # (self vs other)
# # armpit	-0.0506	    0.9999	-0.217	0.1158	False
# # chest	    -0.0745	    1.0	    -0.39	0.2411	False
# # feet	    -0.0883	    0.9916	-0.2955	0.1188	False
# # hair	    0.0869	    1.0	    -0.2664	0.4402	False
# # hand	    0.0305	    1.0	    -0.2601	0.321	False
# # knee	    0.1451	    1.0	    -0.4341	0.7243	False
# # mouth	    -0.127	    0.997	-0.4533	0.1994	False
# # neck	    -0.0494	    1.0	    -0.3232	0.2244	False
# # pelvis	-0.1268	    0.7585	-0.3285	0.0749	False

In [11]:
# # 4. is there any difference when females smell themselves vs when they smell others?
# target = (data[
#               (data['type'].isin(['self', 'other'])) &
#               (data['gender'] == 'female')]
#           .copy())
# three_model = ols("""points ~ C(segment) + C(type) + C(location) +
#                C(segment):C(type) + C(segment):C(location) + C(type):C(location) +
#                C(segment):C(type):C(location)""", data=target).fit()
# res4 = anova_lm(three_model, typ=2)
#
# # C(segment):C(type) -> p= 0,043
# # C(segment):C(location) -> p= 7,251
# # C(type):C(location) -> p= 0,478
# # C(segment):C(type):C(location) -> p= 0,881

In [12]:
# # Lets check the above
# target = data[
#     (data['location'].isin(['front', 'back'])) &
#     (data['gender'] == 'female') &
#     (data['type'].isin(['self', 'other']))
#     ].copy()
# tukey = pairwise_tukeyhsd(endog=target['points'],
#                           groups=target['type'] + target['segment'],
#                           alpha=0.05)
# # None

In [13]:
# # 5. Now, lets focus on condition x body, example: where do female prefer to smell themselves on the front, and where on the back
# target = (data[
#               (data['type'] == 'self') &
#               (data['gender'] == 'female') &
#               (data['location'] == 'front')
#               ]
#           .copy())
# tukey = pairwise_tukeyhsd(endog=target['points'],
#                           groups=target['segment'],
#                           alpha=0.05)